In [ ]:
WORKING = 0
CRACK = 1
LCD = 2
NUM_CLASSES = 3

# Creating dummy data

Only run this section if you want to test stuff quickly without dealing with the real dataset.

In [ ]:
import numpy as np

rng = np.random.default_rng(0)

X = rng.integers(low = 0, high = 256, size = (100, 400, 300, 3))
y = rng.integers(low = 0, high = NUM_CLASSES, size = 100).reshape((100, 1))

# Reading data from files

Don't run this section if you want to work with dummy data, it will overwrite variables `X` and `y` from previous section.

In [ ]:
import os

def collect_file_paths(path: str, ext: str) -> list[str]:
    result = []
    
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path):
            result.extend(collect_file_paths(item_path, ext))
        elif item_path.endswith(ext):
            result.append(item_path)
                
    return result
        

In [ ]:
import cv2
import numpy as np

def images_to_array(image_paths: list[str], shrink_factor: int) -> np.ndarray:
    images = []
    
    for image_file in image_paths:
        image = cv2.cvtColor(cv2.imread(image_file), cv2.COLOR_BGR2RGB)
        
        new_size = (
            image.shape[0] // shrink_factor, 
            image.shape[1] // shrink_factor
        )
        image = cv2.resize(image, dsize = new_size, interpolation = cv2.INTER_CUBIC)
        images.append(image)
        
    return np.array(images)

In [ ]:
file_paths = collect_file_paths("images_for_model", ".jpeg")

input_filepaths = []
labels_list = []

for path in file_paths:
    file_name = path.split("/")[-1]
    labels = file_name.split("_")
    
    if not ("unknown" in labels):
        if "crack" in labels:
            labels_list.append(CRACK)
        elif "lcd" in labels:
            labels_list.append(LCD)
        else:
            labels_list.append(WORKING)
            
        input_filepaths.append(path)
        
X = images_to_array(input_filepaths, 20)
y = np.array(labels_list).reshape((len(labels_list), 1))

# Data preparation

In [ ]:
NUM_SAMPLES = X.shape[0]
IMAGE_WIDTH = X.shape[1]
IMAGE_HEIGHT = X.shape[2]
IMAGE_SHAPE = (X.shape[1], X.shape[2], X.shape[3])

print("Entire dataset")
print("--------------")
print("Input shape: ", X.shape)
print("Output shape: ", y.shape)
print("Number of cracked phones: ", np.sum(y == CRACK))
print("Number of phones with damaged lcd: ", np.sum(y == LCD))
print("Number of working phones: ", np.sum(y == WORKING))
print("")

In [ ]:
from sklearn.model_selection import train_test_split

# TODO: augment LCD images
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print("Training data")
print("-------------")
print("Number of cracked phones: ", np.sum(y_train == CRACK))
print("Number of phones with damaged lcd: ", np.sum(y_train == LCD))
print("Number of working phones: ", np.sum(y_train == WORKING))
print("")

print("Testing data")
print("-------------")
print("Number of cracked phones: ", np.sum(y_test == CRACK))
print("Number of phones with damaged lcd: ", np.sum(y_test == LCD))
print("Number of working phones: ", np.sum(y_test == WORKING))

# Defining the model architecture

In [ ]:
from keras import Input, Model, regularizers, optimizers, Sequential
from keras.layers import Conv2D, Activation, Flatten, Dense, MaxPooling2D, BatchNormalization, Dropout

# This Convolutional neural network architecture is taken from
# Zeiler et al., Visualizing and Understanding Convolutional Networks (2013)
def define_model(learning_rate = 0.001):
    inputs = Input(shape = IMAGE_SHAPE)
    
    hidden = Conv2D(
        filters = 96, 
        kernel_size = (7, 7), 
        strides = (2, 2),
        activation = "relu"
    )(inputs)
    
    hidden = MaxPooling2D(
        pool_size = (3, 3), 
        strides = (2, 2)
    )(hidden)
    
    # Instance (or Contrast) Normalization
    # https://medium.com/techspace-usict/normalization-techniques-in-deep-neural-networks-9121bf100d8
    # https://stackoverflow.com/questions/68088889/how-to-add-instancenormalization-on-tensorflow-keras
    hidden = BatchNormalization(
        axis = 3
    )(hidden, training = True)

    hidden = Conv2D(
        filters = 256, 
        kernel_size = (5, 5), 
        strides = (2, 2),
        activation = "relu"
    )(hidden)

    hidden = MaxPooling2D(
        pool_size = (3, 3), 
        strides = (2, 2)
    )(hidden)
    
    hidden = BatchNormalization(
        axis = 3
    )(hidden, training = True)
  
    hidden = Conv2D(
        filters = 384, 
        kernel_size = (3, 3), 
        strides = (1, 1),
        padding = "same",
        activation = "relu"
    )(hidden)
    
    hidden = Conv2D(
        filters = 384, 
        kernel_size = (3, 3), 
        strides = (1, 1),
        padding = "same",
        activation = "relu"
    )(hidden)
    
    hidden = Conv2D(
        filters = 256, 
        kernel_size = (3, 3), 
        strides = (1, 1),
        padding = "same",
        activation = "relu"
    )(hidden)
    
    hidden = MaxPooling2D(
        pool_size = (3, 3), 
        strides = (2, 2)
    )(hidden)
    
    hidden = BatchNormalization(
        axis = 3
    )(hidden, training = True)
    
    hidden = Flatten()(hidden)
    
    hidden = Dense(4096, activation='relu')(hidden)
    hidden = Dropout(0.5)(hidden)
    hidden = Dense(4096, activation='relu')(hidden)
    hidden = Dropout(0.5)(hidden)
    outputs = Dense(NUM_CLASSES, activation='softmax')(hidden)
    
    model = Model(inputs = inputs, outputs = outputs)

    model.compile(
        loss = 'sparse_categorical_crossentropy', 
        optimizer = optimizers.Adam(learning_rate = learning_rate), 
        metrics = ['accuracy']
    )
    
    return model

# Utility function to plot training history

In [ ]:
import matplotlib.pyplot as plt

def plot_curves(history):
    plt.figure(figsize=(16, 6))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Training', 'Validation'])
    plt.title('Accuracy')
    
    plt.show()

# Training the model and plotting the results

In [ ]:
model = define_model()

history = model.fit(
    x = X_train, y = y_train, 
    validation_data = (X_test, y_test), 
    batch_size = 64, epochs = 10
)

In [ ]:
acc = np.max(history.history['val_accuracy'])
loss = np.min(history.history['val_loss'])

print("accuracy: ", acc)
print("loss: ", loss)

plot_curves(history)

# Saving the trained model

In [ ]:
model_folder_name = "cnn_model_acc=" + str(np.round(acc, 3)) + "_loss=" + str(np.round(acc, 3))
model.save("saved_models/" + model_folder_name)